In [19]:
import auxiliary_tools
import pandas as pd
from geopy.distance import geodesic
import matplotlib.path as mplPath
import numpy as np
from tqdm import tqdm
tqdm.pandas()

/usr/local/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
df_rides, df_others = auxiliary_tools.load_dataset()
df_rides = df_rides.reset_index(drop=True)

# proposals:  123402
# requests 121427
# rides 46616


In [14]:
df_rides = df_rides[df_rides.DAY == '31']
len(df_rides)

3594

In [15]:
df_rides[df_rides.VAN_ID == '23'][['LATITUDE', 'LONGITUDE']].reset_index(drop=True).to_excel("compare_corner_via.xlsx")

In [16]:
df_virtual_stops = pd.read_pickle("df_virtual_stops.pkl")
df_virtual_stops.columns = map(str.upper, df_virtual_stops.columns)
virtual_stops_list = list(zip(df_virtual_stops['LATITUDE'].tolist(), df_virtual_stops['LONGITUDE'].tolist()))

In [17]:
def get_nearst_point(coordinate, list_virtual_points, distance):
    aux = geodesic(kilometers=distance).destination(coordinate, -180)
    left = [aux.latitude, aux.longitude]
    aux = geodesic(kilometers=distance).destination(coordinate, 0)
    right = [aux.latitude, aux.longitude]
    aux = geodesic(kilometers=distance).destination(coordinate, 45)
    c_u_left = [aux.latitude, aux.longitude]
    aux = geodesic(kilometers=distance).destination(coordinate, 135)
    c_u_right = [aux.latitude, aux.longitude]
    aux = geodesic(kilometers=distance).destination(coordinate, 225)
    c_d_left = [aux.latitude, aux.longitude]
    aux = geodesic(kilometers=distance).destination(coordinate, 315)
    c_d_right = [aux.latitude, aux.longitude]
    coordinate_polygon = [c_u_left, c_u_right, c_d_left, c_d_right, c_u_left]
    polygon_path = mplPath.Path(coordinate_polygon)
    
    count_in = 0
    nearest_virtual_stop = [np.nan, np.nan]
    virtual_stop_distance_min = 999999
    for virtual_stop in list_virtual_points:
        if polygon_path.contains_point(virtual_stop):
            count_in+=1
            virtual_stop_found_distance = auxiliary_tools.getGeoDistanceETA_OSRM(coordinate, virtual_stop, 5001, 'walking')[0]
            if virtual_stop_found_distance < virtual_stop_distance_min:
                virtual_stop_distance_min = virtual_stop_found_distance
                nearest_virtual_stop = virtual_stop
    
    nearest_virtual_stop = list(nearest_virtual_stop)
    nearest_virtual_stop.extend([virtual_stop_distance_min])
    return nearest_virtual_stop

def get_route_distance(individual, penalty_const):
    ori_dest = [(first, second) for first, second in zip(individual, individual[1:])]
    penalty_sum = 0
    for pair in ori_dest:
        if pair[0] != pair[1]:
            penalty_sum+=penalty_const
    o_fitness = []
    for od_r in ori_dest:
        o_fitness.append(auxiliary_tools.getGeoDistanceETA_OSRM(od_r[0], od_r[1], 5000, 'driving'))
        
    fitness_value = np.sum([f[0] for f in o_fitness]) + penalty_sum
    return fitness_value

In [20]:
df_rides[['VS_LATITUDE', 'VS_LONGITUDE', 'WALKING_DISTANCE']] = df_rides.progress_apply(lambda row: pd.Series(get_nearst_point([row['LATITUDE'], row['LONGITUDE']], virtual_stops_list, 0.5)), axis=1)

100%|██████████| 3594/3594 [23:29<00:00,  2.55it/s]


In [7]:
df_rides = pd.read_pickle("corner_to_cornerd_client_result.pkl")

In [19]:
df_rides.to_excel("compare_corner_via.xlsx")

In [21]:
results_data = []
for van_id in tqdm(set(df_rides.VAN_ID)):
    df_van = df_rides[df_rides.VAN_ID == van_id].sort_values(by='TIMESTAMP').reset_index(drop=True)
    locations_list = list(zip(df_van['VS_LATITUDE'].tolist(), df_van['VS_LONGITUDE'].tolist()))
    client_cost = df_van.WALKING_DISTANCE.sum()
    operator_cost = get_route_distance(locations_list, 0)
    results_data.append([van_id, 'CURRENT' , client_cost, operator_cost])

100%|██████████| 39/39 [00:21<00:00,  1.82it/s]


In [9]:
# import pickle
# pickle.dump(results_data, open("corner_results_data_list.pkl", "wb"))

In [9]:
import pickle
results_data = pickle.load(open("corner_results_data_list.pkl", "rb"))

In [22]:
df_result_corner = pd.DataFrame(results_data, columns=['VAN_ID', "METHOD", "CLIENT_COST", "OPERATOR_COST"])

In [23]:
df_result_corner.sort_values(by='VAN_ID')

,VAN_ID,METHOD,CLIENT_COST,OPERATOR_COST
22,23,CURRENT,5.7853,209.6799
1,24,CURRENT,6.4758,264.5965
23,25,CURRENT,7.9152,281.3477
34,26,CURRENT,8.8649,285.9133
33,27,CURRENT,6.5178,250.9239
26,28,CURRENT,5.1331,224.4193
10,29,CURRENT,5.5117,213.1649
16,30,CURRENT,7.0897,260.6977
29,31,CURRENT,7.1007,272.6038
9,32,CURRENT,6.8137,243.4607


In [24]:
df_result_corner.to_pickle("../df_result_corner_31.pkl")

In [11]:
# import pickle
# df_rides.to_pickle("corner_to_cornerd_client_result.pkl")